In [ ]:
import matplotlib.pyplot as plt
import torch 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random
from torch.autograd import Variable
import torch.utils.data as Data
import pandas as pd
import torchvision

from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset


In [ ]:
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True # keep True if all the input have same size.
SEED=42
seed_everything(SEED=SEED)

In [ ]:
transform1 = transforms.Compose(
    [transforms.Resize((256, 256)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform2 = transforms.Compose(
    [transforms.Resize((256, 256)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
train_data = torchvision.datasets.ImageFolder('./Caltech101/train',transform=transform1)
eval_data=torchvision.datasets.ImageFolder('./Caltech101/eval',transform=transform1)
test_data=torchvision.datasets.ImageFolder('./Caltech101/test',transform=transform1)

train_loader=Data.DataLoader(dataset=train_data,batch_size=64,
                             shuffle=True, num_workers=2)
eval_loader=Data.DataLoader(dataset=eval_data,batch_size=128,
                             shuffle=True, num_workers=2)
test_loader=Data.DataLoader(dataset=test_data,batch_size=128,
                             shuffle=True, num_workers=2)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(
                in_channels=3,out_channels=64,kernel_size=6,stride=1,
                padding=(1,1)  # if stride=1, padding=(kernel_size-1)/2
            ), 
            nn.ReLU(),            
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2)
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(64,128,6,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )

        
        self.conv3=nn.Sequential(
            nn.Conv2d(128,256,6,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2)
        )   
        self.conv4=nn.Sequential(
            nn.Conv2d(256,256,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2)
        )
        self.conv5=nn.Sequential(
            nn.Conv2d(256,128,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )
        
        self.dense1=nn.Sequential(
            nn.Linear(6272,1024),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.dense2=nn.Sequential(
            nn.Linear(1024,512),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        
        self.dense3=nn.Sequential(
            nn.Linear(512,256),
            nn.Dropout(0.5),
            nn.ReLU()
        )
       
        self.out=nn.Linear(256,101)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.conv5(x)
        x=x.view(x.size(0),-1)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        output=self.out(x)

        return output

cnn = CNN()
print(cnn)  # net architecture

In [ ]:
EPOCH = 30
BATCH_SIZE=32
LR=0.001
stepsize=40
from torch.optim.lr_scheduler import ReduceLROnPlateau
optimizer=torch.optim.Adam(cnn.parameters(),lr=LR,weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=200, factor=0.01)
loss_func=nn.CrossEntropyLoss()



In [ ]:
losses=[]
train_losses=[]
eval_accs=[]
train_accs=[]
stepsize=40
for epoch in range(EPOCH):
    cnn.train()
    
    for step,(x,y) in enumerate(train_loader):
        b_x=Variable(x).cuda()
        b_y=Variable(y).cuda()
        output=cnn(b_x)
        loss=loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step%stepsize==0:
            losses.append(loss.data)

    torch.cuda.empty_cache()
    cnn.eval()
    with torch.no_grad():
        eval_acc=[]
        eval_loss=[]
        for x,y in eval_loader:
            
            b_x=Variable(x).cuda()
            b_y=Variable(y).cuda()        
            output=cnn(b_x)
            loss1=loss_func(output,b_y)
            pred_y=torch.max(output,1)[1].cuda().data.squeeze()
            accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
            eval_acc.append(accuracy)
            scheduler.step(loss1)
            eval_loss.append(loss1.data)
        eval_accs.append(np.mean(eval_acc))
        train_acc=[]
        for x,y in train_loader:
            
            b_x=Variable(x).cuda()
            b_y=Variable(y).cuda()        
            output=cnn(b_x)
            loss2=loss_func(output,b_y)
            pred_y=torch.max(output,1)[1].cuda().data.squeeze()
            accuracy=sum(pred_y==b_y).cpu().numpy()/b_y.size(0)
            train_acc.append(accuracy)
            train_losses.append(loss2.data)
        train_accs.append(np.mean(train_acc))
        
    print('Epoch: ',epoch,'| loss: ',loss.data,'| Eval_acc: ', np.mean(eval_acc),'| Train_acc: ', np.mean(train_acc))
   

In [ ]:
torch.save(cnn, 'epoch 30 batch 32 accuracy 0.69 model.pkl') 
# 保存网络中的参数, 速度快，占空间少
torch.save(cnn.state_dict(),'epoch 30 batch 32 accuracy 0.69 parameters.pkl')

In [ ]:

x=np.arange(len(losses))*stepsize
plt.plot(x,losses)
plt.title('loss ')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.plot(x,train_accs,label='train')
plt.plot(x,eval_accs,label='validate')
plt.title('Accuracy')
plt.xlabel('Step')
plt.ylabel('Accuracy')
plt.legend()
plt.show()